In [1]:
import torch as ch

In [3]:
path = "/home/soroush1/projects/def-kohitij/soroush1/training_fast_publish_faster/weights/aa43d53a-6c1a-4ca0-b9e5-b33c4b2d2f9a/checkpoint_epoch_70_0.64.pth"
checkpoint = ch.load(path)

In [4]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict'])